In [1]:
# -- import from the airtable database --
#  https://airtable.com/
# 

    Database specification
    ---------------------
    
    
    Table Materials
        Name (string), unique
        [tags] (list of string)
        [ChemicalFormula] (string)

    Table Values
        id
        material_id
        property_id
        value (float)
        [source] (string)

    Table Properties
        symbol (string), unique
        unit (string)
  

In [2]:
import json

from airtable import airtable
# https://github.com/nicocanali/airtable-python
# https://github.com/nicocanali/airtable-python/issues/10

In [3]:
# get the api-key and database id (see in the api help example) from json config file
#  airtable-connection.json   configuration file :
#    { "apikey":"MY--KEY",
#      "baseid":"MY--BASE--ID" }


airtableconnect = json.load( open('airtable-connection.json') )
at = airtable.Airtable(airtableconnect['baseid'], airtableconnect['apikey'])

In [4]:
# Download the tables
materials_table = at.get('Materials')
properties_table = at.get('Properties')
values_table = at.get('Values')

In [5]:
%load_ext autoreload
%autoreload 2

import materiallibrary as ml
import pythontools as pytool

In [6]:
# Create the Library
library = ml.Library()

In [7]:
# == import Properties ==
# Add properties to the Library

id2symbol_props = {}
for record in properties_table['records']:
    
    symbol = record['fields']['Symbol']
    id2symbol_props[ record['id'] ] = symbol
    
    prop = ml.Property( **record['fields'] )  # create the Property Object
    library.addproperty( prop )
    
print( '%i physical properties added' % len(library.properties) )

3 physical properties added


In [8]:
library.properties

{'Cp': <Prop. Cp, Chaleur massique (J/kg/K)>,
 'k': <Prop. k, conductivité thermique (W/m/K)>,
 'rho': <Prop. rho, masse volumique (kg/m3)>}

In [9]:
id2symbol_props

{'rec7mCnR2lxb3p30a': 'rho',
 'recPmpBZwRGsR6KmA': 'k',
 'rech3z2J0qGTTFwnh': 'Cp'}

In [10]:
# == import Values ==
# populate a dict matid -> list of values

def get_propertyobject( airtable_id ):
    """ return the property object from the 'airtable' id
    """
    key = id2symbol_props[airtable_id]
    return library.properties[ key ]


sortedvalues = {}
for valueinfo in values_table['records']:
    valueid = valueinfo['id']
    fields = valueinfo['fields']
    
    matid = fields['Material'][0]
    
    # create the Value object
    value = ml.Value( value=fields['value'],
                                   source=fields['Source'],
                                   phiproperty=get_propertyobject( fields['Property'][0] )
                                 )
    
    pytool.dictappend( sortedvalues, matid, value )


In [11]:
# == import Materials ==
for materialinfo in materials_table['records']:
    matid = materialinfo['id']
    fields = materialinfo['fields']

    # create Material Object
    mat = ml.Material( name=fields['Name'], # mandatory
                       values=sortedvalues.get( matid ),
                       tags=fields.get( 'tags' ),
                       chemicalcomp=fields.get( 'ChemicalComposition' ) )
    
    # Append the material to Library
    library.addmaterial( mat )

In [12]:
library.materials

{'Air': <Mat. Air>,
 'Argile': <Mat. Argile>,
 'Bois léger': <Mat. Bois léger>,
 'Briques pleines': <Mat. Briques pleines>,
 'Béton': <Mat. Béton>,
 'Calcaire': <Mat. Calcaire>,
 'Cuivre': <Mat. Cuivre>,
 'Eau': <Mat. Eau>,
 'Sable': <Mat. Sable>,
 'Verre': <Mat. Verre>,
 'notag': <Mat. notag>}

In [13]:
air = library.materials['Air']
cuivre = library.materials['Cuivre']

In [14]:
cuivre.__dict__

{'Cp': 380.0,
 'Cp_min': 380,
 'chemicalcomp': 'Cu',
 'k': 380.0,
 'k_min': 380,
 'name': 'Cuivre',
 'properties': {<Prop. Cp, Chaleur massique (J/kg/K)>,
  <Prop. k, conductivité thermique (W/m/K)>,
  <Prop. rho, masse volumique (kg/m3)>},
 'rho': 8900.0,
 'rho_min': 8900,
 'tags': ['pur (atomique)'],
 'values': [<Val. k=380 W/m/K>, <Val. rho=8900 kg/m3>, <Val. Cp=380 J/kg/K>]}

In [15]:
cuivre.rho

8900.0